In [ ]:
import random
import json
import os
import pickle
 #######
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin


 ###############
#with open(f'Pickle_RL_Model.pkl', 'rb') as f:
    #model = pickle.load(f)
##################


#################

####### 
import torch

from model import NeuralNet
from nltk_utils import bag_of_words, tokenize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)
  #  FILE = "data.pth"
with open(f'Pickle_RL_Model.pkl', 'rb') as f:
    FILE = pickle.load(f)
app = Flask(__name__)
CORS(app)
@app.route("/test", methods=())
def classify():

    
        data = torch.load(FILE)
        input_size = data["input_size"]
        hidden_size = data["hidden_size"]
        output_size = data["output_size"]
        all_words = data['all_words']
        tags = data['tags']
        model_state = data["model_state"]

        model = NeuralNet(input_size, hidden_size, output_size).to(device)
        model.load_state_dict(model_state)
        model.eval()
        bot_name = "Sam"
        print("Let's chat! (type 'quit' to exit)")

        while True:

            # sentence = "do you use credit cards?"
            sentence = input("You: ")
            if sentence == "quit":
                break

            sentence = tokenize(sentence)
            X = bag_of_words(sentence, all_words)
            X = X.reshape(1, X.shape[0])
            X = torch.from_numpy(X).to(device)

            output = model(X)
            _, predicted = torch.max(output, dim=1)

            tag = tags[predicted.item()]

            probs = torch.softmax(output, dim=1)
            prob = probs[0][predicted.item()]
            if prob.item() > 0.75:
                for intent in intents['intents']:
                    if tag == intent["tag"]:
                        print(f"{bot_name}: {random.choice(intent['responses'])}")
            else:
                print(f"{bot_name}: I do not understand...")








           # res =  {random.choice(intent['responses'])}
            ################

           # response = jsonify(res)
            return "Hello"

# running REST interface, port=5000 for direct test, port=5001 for deployment from PM2
if __name__ == "__main__":
        host = os.environ.get("IP","0.0.0.0")
        port= int(os.environ.get('PORT',5001))
        app.run(host=host,port=port)

In [ ]:
jsonify({random.choice(intent['responses'])})

In [ ]:
from flask import Flask

import os
app = Flask(__name__)
@app.route("/")
def hello():
     return "Hello world" 
host = os.environ.get("IP","0.0.0.0")
port= int(os.environ.get('PORT',8000))
app.run(host=host,port=port)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.7:8000/ (Press CTRL+C to quit)
192.168.1.7 - - [19/May/2021 21:51:54] "GET / HTTP/1.1" 200 -
192.168.1.7 - - [19/May/2021 21:51:57] "GET /favicon.ico HTTP/1.1" 404 -
